# Evaluating sarcasm
This notebook will take querry / response pairs that were generated by one of our trained models and then give it to llama model to score for sarcasm

In [7]:
# Imports
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, pipeline
import torch
from tqdm import tqdm
import pandas as pd
import json

In [8]:
# hf login
from huggingface_hub import login
from google.colab import userdata
login(token=userdata.get('HF_TOKEN'))

In [38]:
# Globals
dataset_path = "marcbishara/gst_collection_of_responses"
evaluator_model_name = "gpt-5.1-2025-11-13" #"meta-llama/Llama-4-Maverick-17B-128E-Instruct"
split_name="train"
batch_size = 25

In [39]:
def build_query_response_dataset(
    dataset_name,
    split_name,
    min_text_length=10,
    num_of_rows=None
):

    template_strs = {"</PARENT>\n<RESPONSE>", "<RESPONSE>", "</RESPONSE>", "<PARENT>", "</PARENT>"}
    reply_col_names = ["gpt2_reply", "sft_reply", "ppo_reply", "ppo2_reply", "reddit_reply"]

    ds = load_dataset(dataset_name, split=split_name)

    # Filter out short comments
    ds = ds.filter(lambda x: len(x["parent_comment"]) >= min_text_length)
    ds = ds.filter(lambda x: len(x["gpt2_reply"]) >= min_text_length)
    ds = ds.filter(lambda x: len(x["sft_reply"]) >= min_text_length)
    ds = ds.filter(lambda x: len(x["ppo_reply"]) >= min_text_length)
    ds = ds.filter(lambda x: len(x["ppo2_reply"]) >= min_text_length)

    # Limit by number of rows if provided
    if num_of_rows is not None:
        ds = ds.select(range(num_of_rows))

    def apply_template(sample):

      # Filter out the template strings from parent_comment and reply
      for t in template_strs:
        sample["parent_comment"] = sample["parent_comment"].replace(t, "")
        sample["gpt2_reply"] = sample["gpt2_reply"].replace(t, "")
        sample["sft_reply"] = sample["sft_reply"].replace(t, "")
        sample["ppo_reply"] = sample["ppo_reply"].replace(t, "")
        sample["ppo2_reply"] = sample["ppo2_reply"].replace(t, "")


      '''
      Given the following comment and response found on reddit. Provide a score sarcasm score rating for the response from 0 to 1.
      Highly sarcastic responses should be short, witty and contextually relevant.
      Your response MUST BE FLOAT VALUE FROM 0 TO 1 AND NOTHING ELSE.
      Comment: ...
      Response: ...
      '''
      # templated_query_resp = f"""Given the following comment and response found on reddit. \
      #   Provide a score sarcasm score rating for the response from 0 to 1. \
      #   Highly sarcastic responses should be short, witty and contextually relevant. \
      #   Your response MUST BE FLOAT VALUE FROM 0 TO 1 AND NOTHING ELSE. \
      #   Comment: {sample['parent_comment']} \
      #   Response: {sample['gst_reply']}"""

      for reply_type in reply_col_names:
        templated_query_resp = f"Comment: {sample['parent_comment']} Response: {sample[reply_type]}"
        sample[f'templated_{reply_type}'] = templated_query_resp

      return sample

    # Apply tokenization
    ds = ds.map(apply_template, batched=False)

    # Convert to torch tensors
    ds.set_format(type="torch")

    return ds

In [56]:
# load dataset
dataset = build_query_response_dataset(dataset_name=dataset_path, split_name=split_name, num_of_rows=None) #If you don't want to run the full dataset, limit the number of rows

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [57]:
dataset[0]

{'gpt2_reply': '\ue0b7 <DIV><DIV><DIV> <DIV><DIV><DIV> <DIV><DIV><DIV><DIV><DIV><DIV> <DIV',
 'sft_reply': "~~should've added a link to this post on reddit~~ ",
 'ppo_reply': "~~sigh~~ Golden Eagle is the only possible way to try if you're a big person! ",
 'ppo2_reply': '~~TOMLHHOOOOOM~~! ',
 'parent_comment': ' Is This Legit? Golden Eagle Discounts?  ',
 'reddit_reply': 'I would stay away from it.',
 'templated_gpt2_reply': 'Comment:  Is This Legit? Golden Eagle Discounts?   Response: \ue0b7 <DIV><DIV><DIV> <DIV><DIV><DIV> <DIV><DIV><DIV><DIV><DIV><DIV> <DIV',
 'templated_sft_reply': "Comment:  Is This Legit? Golden Eagle Discounts?   Response: ~~should've added a link to this post on reddit~~ ",
 'templated_ppo_reply': "Comment:  Is This Legit? Golden Eagle Discounts?   Response: ~~sigh~~ Golden Eagle is the only possible way to try if you're a big person! ",
 'templated_ppo2_reply': 'Comment:  Is This Legit? Golden Eagle Discounts?   Response: ~~TOMLHHOOOOOM~~! ',
 'templated_reddi

In [59]:
# Make a dataloader from dataset
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [60]:
i, batch = next(enumerate(dataloader))
print(batch.keys())
print(batch['parent_comment'][0])
print(batch['templated_reddit_reply'][0])
print(batch['templated_gpt2_reply'][0])
print(batch['templated_sft_reply'][0])
print(batch['templated_ppo_reply'][0])
print(batch['templated_ppo2_reply'][0])
print(len(batch['parent_comment']))

dict_keys(['gpt2_reply', 'sft_reply', 'ppo_reply', 'ppo2_reply', 'parent_comment', 'reddit_reply', 'templated_gpt2_reply', 'templated_sft_reply', 'templated_ppo_reply', 'templated_ppo2_reply', 'templated_reddit_reply'])
 Is This Legit? Golden Eagle Discounts?  
Comment:  Is This Legit? Golden Eagle Discounts?   Response: I would stay away from it.
Comment:  Is This Legit? Golden Eagle Discounts?   Response:  <DIV><DIV><DIV> <DIV><DIV><DIV> <DIV><DIV><DIV><DIV><DIV><DIV> <DIV
Comment:  Is This Legit? Golden Eagle Discounts?   Response: ~~should've added a link to this post on reddit~~ 
Comment:  Is This Legit? Golden Eagle Discounts?   Response: ~~sigh~~ Golden Eagle is the only possible way to try if you're a big person! 
Comment:  Is This Legit? Golden Eagle Discounts?   Response: ~~TOMLHHOOOOOM~~! 
25


### Using a remote inference provider

In [61]:
from openai import OpenAI
from google.colab import userdata

# Initialize client with your API key (make sure OPENAI_API_KEY is set in env)
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [62]:
json_schema = {
    "name": "sarcasm_and_fluency_scores_batch",
    "schema": {
        "type": "object",
        "properties": {
            "results": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "Sarcasm score": {"type": "number"},
                        "Fluency score": {"type": "number"}
                    },
                    "required": ["Sarcasm score", "Fluency score"],
                    "additionalProperties": False
                }
            }
        },
        "required": ["results"],
        "additionalProperties": False
    },
    "strict": True
}


system_prompt = f"""Given the following comment and response found on reddit.\
Provide a two scores one score rating the sarcasm of the response and the second score for the fluency of the response.\
Scores should between 0 to 1. When 1 corresponds with high sarcasm and high fluency.\
Highly sarcastic responses should be short, witty and contextually relevant.\
Return an array of {batch_size} JSON objects, each with Sarcasm score and Fluency score preserving the order to match the order of the input received.
"""

In [63]:
reddit_scores = []
sft_scores = []
gpt2_scores = []
ppo_scores = []
ppo2_scores = []

for i, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
  completion_gpt2 = client.chat.completions.create(
      model=evaluator_model_name,
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": json.dumps(batch['templated_gpt2_reply'][:])},
      ],
      response_format={
          "type": "json_schema",
          "json_schema": json_schema,
      },
  )
  gpt2_scores.append((json.loads(completion_gpt2.choices[0].message.content))["results"])

  completion_sft = client.chat.completions.create(
      model=evaluator_model_name,
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": json.dumps(batch['templated_sft_reply'][:])},
      ],
      response_format={
          "type": "json_schema",
          "json_schema": json_schema,
      },
  )
  sft_scores.append((json.loads(completion_sft.choices[0].message.content))["results"])

  completion_ppo = client.chat.completions.create(
      model=evaluator_model_name,
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": json.dumps(batch['templated_ppo_reply'][:])},
      ],
      response_format={
          "type": "json_schema",
          "json_schema": json_schema,
      },
  )
  ppo_scores.append((json.loads(completion_ppo.choices[0].message.content))["results"])

  completion_ppo2 = client.chat.completions.create(
      model=evaluator_model_name,
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": json.dumps(batch['templated_ppo2_reply'][:])},
      ],
      response_format={
          "type": "json_schema",
          "json_schema": json_schema,
      },
  )
  ppo2_scores.append((json.loads(completion_ppo2.choices[0].message.content))["results"])

  completion_reddit = client.chat.completions.create(
      model=evaluator_model_name,
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": json.dumps(batch['templated_reddit_reply'][:])},
      ],
      response_format={
          "type": "json_schema",
          "json_schema": json_schema,
      },
  )
  reddit_scores.append((json.loads(completion_reddit.choices[0].message.content))["results"])


100%|██████████| 40/40 [17:49<00:00, 26.73s/it]


In [32]:
ppo2_scores

[[{'Sarcasm score': 0.7, 'Fluency score': 0.3},
  {'Sarcasm score': 0.6, 'Fluency score': 0.9},
  {'Sarcasm score': 0.8, 'Fluency score': 0.4},
  {'Sarcasm score': 0.7, 'Fluency score': 0.3},
  {'Sarcasm score': 0.7, 'Fluency score': 0.6},
  {'Sarcasm score': 0.4, 'Fluency score': 0.8},
  {'Sarcasm score': 0.8, 'Fluency score': 0.7},
  {'Sarcasm score': 0.5, 'Fluency score': 0.2},
  {'Sarcasm score': 0.5, 'Fluency score': 0.9},
  {'Sarcasm score': 0.6, 'Fluency score': 0.5},
  {'Sarcasm score': 0.6, 'Fluency score': 0.4},
  {'Sarcasm score': 0.2, 'Fluency score': 0.2},
  {'Sarcasm score': 0.5, 'Fluency score': 0.2},
  {'Sarcasm score': 0.3, 'Fluency score': 0.2},
  {'Sarcasm score': 0.4, 'Fluency score': 0.1},
  {'Sarcasm score': 0.3, 'Fluency score': 0.6},
  {'Sarcasm score': 0.4, 'Fluency score': 0.1},
  {'Sarcasm score': 0.7, 'Fluency score': 0.8},
  {'Sarcasm score': 0.3, 'Fluency score': 0.3},
  {'Sarcasm score': 0.4, 'Fluency score': 0.2},
  {'Sarcasm score': 0.4, 'Fluency score'

In [64]:
def extract_scores(scores_list):
    sarcasm_scores = []
    fluency_scores = []
    for batch_scores in scores_list:
        for item in batch_scores:
            sarcasm_scores.append(item.get('Sarcasm score'))
            fluency_scores.append(item.get('Fluency score'))
    return sarcasm_scores, fluency_scores

# Example usage:
gpt2_sarcasm, gpt2_fluency = extract_scores(gpt2_scores)
sft_sarcasm, sft_fluency = extract_scores(sft_scores)
ppo_sarcasm, ppo_fluency = extract_scores(ppo_scores)
ppo2_sarcasm, ppo2_fluency = extract_scores(ppo2_scores)
reddit_sarcasm, reddit_fluency = extract_scores(reddit_scores)

In [65]:
# len(ppo2_sarcasm)
len(dataset)

1000

In [66]:
results_df_data = []
num_rows_to_include = len(dataset)

for i in range(num_rows_to_include):
    row_data = {
        'parent_comment': dataset[i]['parent_comment'],
        'reddit_reply': dataset[i]['reddit_reply'],
        'gpt2_reply': dataset[i]['gpt2_reply'],
        'sft_reply': dataset[i]['sft_reply'],
        'ppo_reply': dataset[i]['ppo_reply'],
        'ppo2_reply': dataset[i]['ppo2_reply'],

        'reddit_sarcasm': reddit_sarcasm[i],
        'reddit_fluency': reddit_fluency[i],
        'gpt2_sarcasm': gpt2_sarcasm[i],
        'gpt2_fluency': gpt2_fluency[i],
        'sft_sarcasm': sft_sarcasm[i],
        'sft_fluency': sft_fluency[i],
        'ppo_sarcasm': ppo_sarcasm[i],
        'ppo_fluency': ppo_fluency[i],
        'ppo2_sarcasm': ppo2_sarcasm[i],
        'ppo2_fluency': ppo2_fluency[i],
    }
    results_df_data.append(row_data)

results_df = pd.DataFrame(results_df_data)

In [67]:
reply_types = ['reddit', 'gpt2', 'sft', 'ppo', 'ppo2']

print("--- Average and Standard Deviation of Scores ---\n")

for reply_type in reply_types:
    sarcasm_col = f'{reply_type}_sarcasm'
    fluency_col = f'{reply_type}_fluency'

    avg_sarcasm = results_df[sarcasm_col].mean()
    std_sarcasm = results_df[sarcasm_col].std()

    avg_fluency = results_df[fluency_col].mean()
    std_fluency = results_df[fluency_col].std()

    print(f"Reply Type: {reply_type.upper()}:")
    print(f"  Sarcasm Score: Avg = {avg_sarcasm:.2f}, Std = {std_sarcasm:.2f}")
    print(f"  Fluency Score: Avg = {avg_fluency:.2f}, Std = {std_fluency:.2f}\n")

--- Average and Standard Deviation of Scores ---

Reply Type: REDDIT:
  Sarcasm Score: Avg = 0.51, Std = 0.30
  Fluency Score: Avg = 0.91, Std = 0.09

Reply Type: GPT2:
  Sarcasm Score: Avg = 0.14, Std = 0.14
  Fluency Score: Avg = 0.26, Std = 0.16

Reply Type: SFT:
  Sarcasm Score: Avg = 0.40, Std = 0.20
  Fluency Score: Avg = 0.45, Std = 0.22

Reply Type: PPO:
  Sarcasm Score: Avg = 0.44, Std = 0.20
  Fluency Score: Avg = 0.44, Std = 0.21

Reply Type: PPO2:
  Sarcasm Score: Avg = 0.37, Std = 0.20
  Fluency Score: Avg = 0.43, Std = 0.22



## Save dataset to huggingface

In [69]:
# hugging face login
from huggingface_hub import login
from google.colab import userdata
login(token=userdata.get('HF_TOKEN'))

In [68]:
# save df to csv
filename = f"generated_sarcastic_replies_with_scores_{evaluator_model_name}_Len-{len(dataset)}.csv"
results_df.to_csv(f"/content/{filename}", index=False)

In [70]:
# push scv to huggingface
from huggingface_hub import upload_file
upload_file(
    path_or_fileobj="/content/generated_sarcastic_replies_with_scores.csv",
    path_in_repo=f"{filename}",
    repo_id="marcbishara/gst_collection_of_responses",
    repo_type="dataset",
    token=True,
)

CommitInfo(commit_url='https://huggingface.co/datasets/marcbishara/gst_collection_of_responses/commit/b8090f71109c23103eaa5d5a7cb3af504a361ee9', commit_message='Upload generated_sarcastic_replies_with_scores_gpt-5.1-2025-11-13_Len-1000.csv with huggingface_hub', commit_description='', oid='b8090f71109c23103eaa5d5a7cb3af504a361ee9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/marcbishara/gst_collection_of_responses', endpoint='https://huggingface.co', repo_type='dataset', repo_id='marcbishara/gst_collection_of_responses'), pr_revision=None, pr_num=None)

# Archive

#### Hugging face

In [ ]:
from huggingface_hub import InferenceClient
client = InferenceClient()

In [ ]:
dataset = dataset.add_column("fluency score", [0] * len(dataset))
dataset = dataset.add_column("sarcasm score", [0] * len(dataset))

In [ ]:
dataset

Dataset({
    features: ['parent_comment', 'gst_reply', 'templated_msg', 'fluency score', 'sarcasm score'],
    num_rows: 1000
})

In [ ]:
for i in range(10): #len(batch['templated_msg'])
  completion = client.chat.completions.create(
      model= evaluator_model_name,
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": dataset['templated_msg'][i]},
      ],
      response_format={
          "type": "json_schema",
          "json_schema": json_schema,
      },
  )
  print(f"Resp: {dataset['gst_reply'][i]} - Fluency:{json.loads(completion.choices[0].message.content)['Fluency score']}")
  print(f"Sarcasm: {json.loads(completion.choices[0].message.content)['Sarcasm score']}")

Resp: No, I'm just taking a moment to explain why I'm upset.  - Fluency:0.8
Sarcasm: 0.9
Resp: Yes, because it's a big deal and not something we should be doing at all.  - Fluency:0.9
Sarcasm: 0.8
Resp: Oh man, I'm so glad we got to see this in person  - Fluency:0.8
Sarcasm: 0.9
Resp: It's always the best player  - Fluency:0.9
Sarcasm: 0.8
Resp: You forgot the  - Fluency:0.8
Sarcasm: 0.9
Resp: That's why I am so glad the car is still on the road  - Fluency:0.9
Sarcasm: 0.8
Resp: Because the game doesn't have a lot of grenades in it.  - Fluency:1
Sarcasm: 0.8


HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/groq/openai/v1/chat/completions (Request ID: Root=1-6934915e-31366df939fe27be7428edd9;081bc7e1-f72b-43c2-a73e-60c3a209a67d)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

In [ ]:
json_resp =
json_resp['Sarcasm score']

0.9

### Loading a model for inference locally

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(evaluator_model_name)
model = AutoModelForCausalLM.from_pretrained(evaluator_model_name, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/448 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/7.35k [00:00<?, ?B/s]

...

model-00010-of-00055.safetensors:   0%|          | 0.00/21.5G [00:00<?, ?B/s]

model-00015-of-00055.safetensors:   0%|          | 0.00/21.5G [00:00<?, ?B/s]

model-00016-of-00055.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

model-00017-of-00055.safetensors:   0%|          | 0.00/21.5G [00:00<?, ?B/s]

RuntimeError: Data processing error: CAS service error : IO Error: No space left on device (os error 28)

In [ ]:

# build generation config and a small pipeline wrapper for easier inference
gen_cfg = GenerationConfig(
    temperature= 0.9,
    min_new_tokens= 3,
    top_p= 0.9,
    top_k= 0,
    do_sample= True,
    pad_token_id= tokenizer.eos_token_id, # most decoder models don't have a padding token - use EOS token instead
    max_new_tokens= 10, # specify how many tokens you want to generate at most
)

text_gen = pipeline(
    "text-generation",
    model=evaluator_model_name,
    tokenizer=tokenizer,
    device='cuda',
)

def generate_reply(parent_comment: str) -> str:
    out = text_gen(parent_comment, generation_config=gen_cfg, return_full_text=False, clean_up_tokenization_spaces=True)
    # pipeline returns list of dicts; take first
    return out[0]["generated_text"].strip()

config.json:   0%|          | 0.00/874 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

Device set to use cuda


In [ ]:
# generation for PPO
text_gen_ppo = pipeline(
    "text-generation",
    model=ppo_model_name,
    tokenizer=tokenizer,
    device='cuda',
    revision=ppo_revision
)

def generate_reply_ppo(parent_comment: str) -> str:
    out = text_gen_ppo(parent_comment, generation_config=gen_cfg, return_full_text=False, clean_up_tokenization_spaces=True)
    # pipeline returns list of dicts; take first
    return out[0]["generated_text"].strip()

config.json:   0%|          | 0.00/874 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of the model checkpoint at marcbishara/GenerallySarcasticTransformer were not used when initializing GPT2LMHeadModel: ['v_head.summary.bias', 'v_head.summary.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

Device set to use cuda


In [ ]:
all_results = []

for i, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
    print(f"Processing batch {i+1}/{len(dataloader)}")

    # Get all queries in the batch
    queries = batch['query']

    # Process entire batch through pipeline at once
    replies = text_gen(queries, generation_config=gen_cfg, return_full_text=False, clean_up_tokenization_spaces=True)

    # Extract generated text from replies (handle list structure)
    generated_texts = [reply[0]["generated_text"].strip() for reply in replies]

    # Add all results to list
    for parent_comment, reply_text in zip(queries, generated_texts):
        all_results.append({
            "parent_comment": parent_comment,
            "gst_reply": reply_text
        })

    # Print a sample from the batch for sanity check
    print(f"Sample Parent Comment: {queries[0]}")
    print(f"Generated Reply: {generated_texts[0]}")

# Create dataframe from all results at once
parent_response = pd.DataFrame(all_results)

# Save to CSV
parent_response.to_csv("/content/generated_sarcastic_replies.csv", index=False)

  0%|          | 0/4 [00:00<?, ?it/s]`generation_config` default values have been modified to match model-specific defaults: {'bos_token_id': 50256, 'eos_token_id': [50256]}. If this is not desired, please set these values explicitly.


Processing batch 1/4


 25%|██▌       | 1/4 [00:54<02:42, 54.13s/it]

Sample Parent Comment: <PARENT> Is This Legit? Golden Eagle Discounts? </PARENT>
<RESPONSE> 
Generated Reply: ~~Criminals~~: So, you're in a holdout? </RESPONSE>
Processing batch 2/4


 50%|█████     | 2/4 [01:45<01:45, 52.68s/it]

Sample Parent Comment: <PARENT> How to Tell Black People Apart by David Alan Grier </PARENT>
<RESPONSE> 
Generated Reply: ~~ there are **black** people on this planet. </RESPONSE>
Processing batch 3/4


 75%|███████▌  | 3/4 [02:39<00:53, 53.32s/it]

Sample Parent Comment: <PARENT> Yes, it is a bit of a regressive tax. </PARENT>
<RESPONSE> 
Generated Reply: ive seen enough libertarians to know that when liberals don't pay taxes, they can't afford to hire people! </RESPONSE>
Processing batch 4/4


100%|██████████| 4/4 [03:28<00:00, 52.05s/it]

Sample Parent Comment: <PARENT> Hey that kangaroo stole my ball </PARENT>
<RESPONSE> 
Generated Reply: ~~yes, because no one is going to ever learn about the niggers </RESPONSE>


In [ ]:
# Generate PPO responses
ppo_all_results = []

for i, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
    print(f"Processing batch {i+1}/{len(dataloader)}")

    # Get all queries in the batch
    queries = batch['query']

    # Process entire batch through pipeline at once
    replies = text_gen_ppo(queries, generation_config=gen_cfg, return_full_text=False, clean_up_tokenization_spaces=True)

    # Extract generated text from replies (handle list structure)
    generated_texts = [reply[0]["generated_text"].strip() for reply in replies]

    # Add all results to list
    for parent_comment, reply_text in zip(queries, generated_texts):
        ppo_all_results.append({
            "parent_comment": parent_comment,
            "ppo_reply": reply_text
        })

    # Print a sample from the batch for sanity check
    print(f"Sample Parent Comment: {queries[0]}")
    print(f"PPO Reply: {generated_texts[0]}")

  0%|          | 0/4 [00:00<?, ?it/s]

Processing batch 1/4


 25%|██▌       | 1/4 [00:55<02:45, 55.11s/it]

Sample Parent Comment: <PARENT> Is This Legit? Golden Eagle Discounts? </PARENT>
<RESPONSE> 
PPO Reply: ~~you've heard this is the one redditor!~~they never account for people who don't purchase the perk!~~ </RESPONSE>
Processing batch 2/4


 50%|█████     | 2/4 [01:48<01:48, 54.17s/it]

Sample Parent Comment: <PARENT> How to Tell Black People Apart by David Alan Grier </PARENT>
<RESPONSE> 
PPO Reply: ~~they're not that far removed from us~~ </RESPONSE>
Processing batch 3/4


 75%|███████▌  | 3/4 [02:41<00:53, 53.51s/it]

Sample Parent Comment: <PARENT> Yes, it is a bit of a regressive tax. </PARENT>
<RESPONSE> 
PPO Reply: ~~REEEEEEEEEE!~~
Processing batch 4/4


100%|██████████| 4/4 [03:29<00:00, 52.39s/it]

Sample Parent Comment: <PARENT> Hey that kangaroo stole my ball </PARENT>
<RESPONSE> 
PPO Reply: ~~Haha!~~ if only he knew how to hold his own! </RESPONSE>


In [ ]:
all_results[0]['gst_reply']

"~~Criminals~~: So, you're in a holdout? </RESPONSE>"

In [ ]:
ppo_all_results[0]['ppo_reply']

"~~you've heard this is the one redditor!~~they never account for people who don't purchase the perk!~~ </RESPONSE>"

In [ ]:
template_strs = {"</PARENT>\n<RESPONSE>", "<RESPONSE>", "</RESPONSE>", "<PARENT>", "</PARENT>"}
# special_ids = torch.tensor(tokenizer.all_special_ids, device='cuda')
all_responses = []
all_querries = []

for i in range(len(all_results)):
  r_str = all_results[i]['gst_reply']
  q_str = all_results[i]['parent_comment']

  for s in template_strs:
      q_str = q_str.replace(s, "")
      r_str = r_str.replace(s, "")

  all_responses.append(r_str)
  all_querries.append(q_str)

In [ ]:
ppo_all_responses = []
ppo_all_querries = []

for i in range(len(ppo_all_results)):
  r_str = ppo_all_results[i]['ppo_reply']
  q_str = ppo_all_results[i]['parent_comment']

  for s in template_strs:
      q_str = q_str.replace(s, "")
      r_str = r_str.replace(s, "")

  ppo_all_responses.append(r_str)
  ppo_all_querries.append(q_str)

In [ ]:
print(ppo_all_responses[0])
print(all_responses[0])

~~you've heard this is the one redditor!~~they never account for people who don't purchase the perk!~~ 
~~Criminals~~: So, you're in a holdout? 
